In [27]:
#[(phase, Vector of string with the Gate name and qubit indices)]

In [28]:
%pip install --user openfermion

Note: you may need to restart the kernel to use updated packages.


In [29]:
%pip install sympy

Note: you may need to restart the kernel to use updated packages.


In [30]:
import pyzx as zx
import sympy as sp

In [31]:
pauli_string = [(1.0,["X0","X1"]),(1.0,["X1","X2"]),(-1.0,["Z0"]),(-1.0,["Z1"]),(-1.0,["Z2"])]

In [ ]:
def w_spider(outputs=1): return zx.generate.spider('W', 1, outputs)

In [67]:
total_qubits = 0

for term in pauli_string:
    # Get the number from the gate names to determine the number of qubits
    qubit_indices = [int(gate[1:]) for gate in term[1]]
    max_index = max(qubit_indices)
    if max_index > total_qubits:
        total_qubits = max_index
    print("Alpha:", term[0], "Gates:", term[1])
    
total_qubits += 1  # Adjust for 0-based indexing
print("Total qubits needed:", total_qubits)

Alpha: 1.0 Gates: ['X0', 'X1']
Alpha: 1.0 Gates: ['X1', 'X2']
Alpha: -1.0 Gates: ['Z0']
Alpha: -1.0 Gates: ['Z1']
Alpha: -1.0 Gates: ['Z2']
Total qubits needed: 3


In [33]:
main_graph = zx.Graph()

root_x_vertex = main_graph.add_vertex(zx.VertexType.X, qubit=0, row=0)
w_input = main_graph.add_vertex(zx.VertexType.W_INPUT, qubit=0, row=0)
w_output = main_graph.add_vertex(zx.VertexType.W_OUTPUT, qubit=0, row=0)
main_graph.add_edge((root_x_vertex, w_input))
main_graph.add_edge((w_input, w_output)) 

w_labels = {w_input: "W_IN", w_output: "W_OUT"}

for q in range(total_qubits):
    in_vertex = main_graph.add_vertex(zx.VertexType.BOUNDARY, qubit=q, row=0)
    
# zx.draw(main_graph, labels=True)

In [34]:
current_row = 1  # Start adding operations from row 1

for term in pauli_string:
    phase = term[0]
    gates = term[1]
    
    for gate in gates:
        gate_type = gate[0]
        qubit_index = int(gate[1:])
        
        row_increment = 0
        
        if gate_type == 'X':
            first_hadamard = main_graph.add_vertex(zx.VertexType.H_BOX, qubit=qubit_index, row=current_row)
            z_vertex = main_graph.add_vertex(zx.VertexType.Z, qubit=qubit_index, row=current_row + 1)
            second_hadamard = main_graph.add_vertex(zx.VertexType.H_BOX, qubit=qubit_index, row=current_row + 2)
            
            main_graph.add_edge((first_hadamard, z_vertex))
            main_graph.add_edge((z_vertex, second_hadamard))
            
            row_increment += 3
            
        elif gate_type == 'Z':
            z_vertex = main_graph.add_vertex(zx.VertexType.Z, qubit=qubit_index, row=current_row)
            row_increment += 1
            
        elif gate_type == 'Y':
            x_vertex_one = main_graph.add_vertex(zx.VertexType.X, qubit=qubit_index, row=current_row, phase=sp.pi/2)
            z_vertex = main_graph.add_vertex(zx.VertexType.Z, qubit=qubit_index, row=current_row + 1)
            x_vertex_two = main_graph.add_vertex(zx.VertexType.X, qubit=qubit_index, row=current_row + 2, phase=-sp.pi/2)
            
            main_graph.add_edge((x_vertex_one, z_vertex))
            main_graph.add_edge((z_vertex, x_vertex_two))
            
            row_increment += 3
            
        print("Adding gate:", gate_type, "on qubit", qubit_index)
        
    current_row += row_increment

Adding gate: X on qubit 0
Adding gate: X on qubit 1
Adding gate: X on qubit 1
Adding gate: X on qubit 2
Adding gate: Z on qubit 0
Adding gate: Z on qubit 1
Adding gate: Z on qubit 2


In [35]:
for q in range(total_qubits):
    out_vertex = main_graph.add_vertex(zx.VertexType.BOUNDARY, qubit=q, row=current_row)

In [ ]:

for q in range(total_qubits):
    #find all vertices for qubit q
    vertices_on_qubit = [v for v in main_graph.vertices() if main_graph.qubit(v) == q]
    edges = main_graph.edge_set()
    
    # run through vertices and check wich are not connected
    for i in range(len(vertices_on_qubit)):

        if i in w_labels:
            continue

        v1 = vertices_on_qubit[i]
        
        if i+1 >= len(vertices_on_qubit):
            break
        
        v2 = vertices_on_qubit[i+1]
        if (v1, v2) not in edges and (v2, v1) not in edges:
            main_graph.add_edge((v1, v2))


zx.draw(main_graph, labels=True)

dict_keys([0, 1, 2, 3, 4])
